## Titanic Lab Competition Entry 2018

Lillian Ellis // Completed 12/9/18

**Abstract:**

In this lab, I participated in the Kaggle Titanic competition where we worked to train a machine to predict who lived and who died on the Titanic. We were given both a training data set and a test data set to train and test our machines! As I found throughout the lab, there were some reoccuring trends for who survived and who did not. Throughout my lab, I learned a lot about how to take analysis from a data set and acctually get to make predictions on it!

## Import Packages

First, I imported all of my dataset packages, along with machine learning packages to use for the lab. 

In [3349]:
#import data packages
import math as m
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns #(sam seaborn!!)
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Input Data Sets

Next, I imported and defined my data sets (both train and test) that I will use throughout the rest of my lab. Throughout my lab, I made sure that everything I did for test, I also did for train, to ensure both data sets contained the same values. 

In [3350]:
#Input csv filed into two data fram object: train_df and test_df
#train = 891 entries of passengers, features, with labels if survived or died 
#test = 418 entries of titanic passengers, features, no labels
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

Note: Throughout my lab, I greatly referred to Manav Sehgal's notebook on the Kaggle competition. In Manav's notebook, he includes the combine data set defined above (test and train combined). I had some troubles with combine throughout my lab, but it was helpful when converting male and female passengers to 0 and 1. Here is the link to his notebook: https://www.kaggle.com/startupsci/titanic-data-science-solutions 

## Check/Analyze Data

First, I wanted to take a clean look at the first 10 lines of data from the train set (which I have printed below) to get a look at what trends I notice off the bat, what the headers are, and what I may need to clean. 

In [3351]:
#Examine first 10 rows to take a look at the data 
#note: passengerID randomly assigned by Kaggle (correlates with index)
train_df[:10]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Most of my data looks pretty clean, but I do notice that there are a few missing values. I next printed the info of train to get an overview of the entire data set. 

**Printing Train Info**

In [3352]:
#print info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


From looking at this, we have lots of data for everything execpt for age (limited knowledge) and cabin (cabin numbers for 1st class). Data is also missing data for 2 passengers on where they embarked. We have fairly solid and consistent data with all of the other columns, which have 891 values each. 

**Printing Test Info**

In [3353]:
#lookin at test set
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


Still lots of missing ages, cabins (lots of missing data!), one fare missing. There are integer, object, and float values. 

Overall:
- both sets have all features, there is nothing off here
- most features are pretty complete exept for age, cabin
- some missing features surrounding embarking location and fare

## Feature Analysis and Clean Up of Data

I conducted some analysis and clean up on the features to determine a good set to work with when training my classifier- based on the features that will tell us the labels (these must be numerical)

Determining:
- which features to get rid of 
- which features to clean
- **which features correlate to labels (and how accurate each one is)**
- how to handle missing data
- maybe we will need to combine data (gender and class? gender and embarked? etc.)

Need to clean up features for both training and test at the same time (so these align and chance at accurate feature is good)

List of Data Cleaning Tasks:
- print basic correlations between passenger, class, sex, SipSp, Embarked, etc. (as these are not missing much data, pretty telling)
- print graphs to see other correlations 
- change female and male to 1 and 0
- combine SibSp and Parch into family 
- Create other values based on finds from correlations

In [3354]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [3355]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False) 

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [3356]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [3357]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [3358]:
train_df[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False) 

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


Citation for correlation tests: https://www.kaggle.com/startupsci/titanic-data-science-solutions lines 9-12 from Manav Sehgal. Throughout my lab, I referred to his notebook, as it was a very helpful way to get started. I only used a few pieces of his code, but referring to his code on data analysis, and changing female and male to integers was really helpful. 

Here I printed some graphs to make the correlation tests above easier to read. 

In [3359]:
#sns.barplot(x = "Family", y = "Survived", data = train_df)

(note: I commented out my first graph, although it worked, because it was using too much memory and it kept erroring out) 

In [3360]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
#grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
#grid.map(plt.hist, 'Age', alpha=.5, bins=20)
#grid.add_legend();

In [3361]:
# grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

/Users/lillianellis/anaconda3/lib/python3.6/site-packages/seaborn/axisgrid.py:230: UserWarning: The `size` paramter has been renamed to `height`; please update your code.
  warnings.warn(msg, UserWarning)
/Users/lillianellis/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/lillianellis/anaconda3/lib/python3.6/site-packages/seaborn/axisgrid.py:715: UserWarning: Using the barplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)


**Main Takeaways from Correlations:**

- sex is the biggest indicator of survival!
    - 74% of females survive , 19% of males survive
- class is also a pretty big indicator 
    - 63% of those in 1st class survived, 47% in 2nd, and 24% in 3rd
- whether you were alone or not is also pretty big!
    - the more family members you had on board, the less likely you were to survive 

So, I decided to especially focus on sex, class, and family members. 

My first step in cleaning the sex column was to convert females to 1 and males to 0. I got this code from Manav's notebook. 

In [3362]:
# changing males to 0 and females to 1 , changing the type to an integer 
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


Then, I combined columns for parents/children and sibling/spouses to just family members- because there is a consistant linear regression: those who had more family members were less likely to survive. 

In [3363]:
#combining Sibling and spouses with parents and children to create a new family column 
train_df["Family"] = train_df["SibSp"] + train_df["Parch"]
test_df["Family"] = test_df["SibSp"] + test_df["Parch"] 

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,0


Next, I deleted  sibsp and parch in both data sets - as I will now instead using new column of family 

In [3364]:
#drop columns that Family replaces 
train_df.drop('Parch', axis = 1, inplace = True)
train_df.drop('SibSp', axis = 1, inplace = True)
test_df.drop('Parch', axis = 1, inplace = True)
test_df.drop('SibSp', axis = 1, inplace = True)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,373450,8.0500,NaN,S,0


However, I then decided to create an "IsAlone" column- which indicates if you have family members or are all alone (because if you are alone- higher chance of survival). This new column simplified data, and eliminated potential outliers. I also got this idea from Manav, but I modified it as I was having trouble with the "combine" data set. 

In [3365]:
test_df["IsAlone"] = 0
train_df["IsAlone"] = 0
train_df.loc[train_df['Family'] == 1, 'IsAlone'] = 1
test_df.loc[test_df['Family'] == 1, 'IsAlone'] = 1

#This was Manav's code (which I modified)
#for dataset in combine:
    #dataset['IsAlone'] = 0
    #dataset.loc[dataset['Family'] == 1, 'IsAlone'] = 1

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            714 non-null float64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Family         891 non-null int64
IsAlone        891 non-null int64
dtypes: float64(2), int64(6), object(4)
memory usage: 83.6+ KB


As you can see below, being alone was a clear advantage, as 34% of those not alone survived, while 55% of those who were alone survived!

In [3366]:
train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.346575
1,1,0.552795


The last step I took before training my machine was deleting all of the columns other than the ones I found were the most relavant, telling, full, and now clean! (IsAlone, Class, and Sex).

In [3367]:
for val in train_df.columns:
    if val != 'IsAlone' and val != 'Pclass' and val != 'Sex' and val != 'Survived':
        train_df = train_df.drop([val], axis=1)
for val in test_df.columns:
    if val != 'IsAlone' and val != 'Pclass' and val != 'Sex':
        test_df = test_df.drop([val], axis=1)

These are what my two data sets now look like:

In [3368]:
train_df.head()

,Survived,Pclass,Sex,IsAlone
0,0,3,0,1
1,1,1,1,1
2,1,3,1,0
3,1,1,1,1
4,0,3,0,0


In [3369]:
test_df.head() 

,Pclass,Sex,IsAlone
0,3,0,0
1,3,1,1
2,2,0,0
3,3,0,0
4,3,1,0


## Training Classifiers

Now that I am done with cleaning, and selected the data I want to keep in both, I am ready to train my classifier. I first created the machine, fed in the feature set to train the machine with my labels, and then started to make predictions with 6 different classifiers and printed the accuracy method! 

TO DO:
- create machine
- feed feature set to train machine with the labels
- start to make predictions and get accuracy rate up! 

In [3370]:
#split training est into the feature list w/o passenger id and labels
# features - x
# labels - y
X_train = train_df.drop(["Survived"], axis=1)
Y_train = train_df["Survived"]

X_train.head()
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [3371]:
#from sklearn.cross_validation import train_test_split
#train_test_split splits our data for us into training and testing! 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

#this means a 70-30 split, pulling out 30% of data for test size (what percent of the data goes into the test)
#4 variables are returned from the method call

valid_X_train, valid_X_test, valid_Y_train, valid_Y_test = train_test_split(X_train, Y_train, train_size = .7, test_size = .3)

valid_X_train.shape, valid_X_test.shape, valid_Y_train.shape, valid_Y_test.shape 
 

((623, 3), (268, 3), (623,), (268,))

In [3372]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(valid_X_train, valid_Y_train)
Y_pred = logreg.predict(valid_X_test)

print(accuracy_score(valid_Y_test, Y_pred)* 100)

#a check to see if data overfitted- if these numbers are close together, then the data is not overfit 
#this uses the method of the used classifier
# I got this specific code from Ms. Sconyers!
print((logreg.score(X_train, Y_train)*100), logreg.score(valid_X_train, valid_Y_train)*100)

82.08955223880598
78.67564534231201 77.20706260032102


In [3373]:
svc = SVC()
svc.fit(valid_X_train, valid_Y_train)
Y_pred = svc.predict(valid_X_test)
#acc_svc = round(svc.score(valid_X_train, valid_Y_train) * 100, 2)
#acc_svc

print(accuracy_score(valid_Y_test, Y_pred)* 100)

#a check to see if data overfitted- if these numbers are close together, then the data is not overfit 
print((svc.score(X_train, Y_train)*100), svc.score(valid_X_train, valid_Y_train)*100)

82.08955223880598
78.67564534231201 77.20706260032102


In [3374]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(valid_X_train, valid_Y_train)
Y_pred = knn.predict(valid_X_test)
#acc_knn = round(knn.score(valid_X_train, valid_Y_train) * 100, 2)
#acc_knn

print(accuracy_score(valid_Y_test, Y_pred)* 100)

#a check to see if data overfitted- if these numbers are close together, then the data is not overfit 
print((knn.score(X_train, Y_train)*100), knn.score(valid_X_train, valid_Y_train)*100)

80.22388059701493
76.65544332210999 75.12038523274478


In [3375]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(valid_X_train, valid_Y_train)
Y_pred = gaussian.predict(valid_X_test)
#acc_gaussian = round(gaussian.score(valid_X_train, valid_Y_train) * 100, 2)
#acc_gaussian

print(accuracy_score(valid_Y_test, Y_pred)* 100)

#a check to see if data overfitted- if these numbers are close together, then the data is not overfit 
print((gaussian.score(X_train, Y_train)*100), gaussian.score(valid_X_train, valid_Y_train)*100)

81.71641791044776
77.89001122334456 76.24398073836277


Both decision tree and random forest consistently print an accuracy score around 80, which is highest out of all of my classifiers. 

In [3376]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(valid_X_train, valid_Y_train)
Y_pred = decision_tree.predict(valid_X_test)
#acc_decision_tree = round(decision_tree.score(valid_X_train, valid_Y_train) * 100, 2)
#acc_decision_tree

print(accuracy_score(valid_Y_test, Y_pred)* 100)

#a check to see if data overfitted- if these numbers are close together, then the data is not overfit 
print((decision_tree.score(X_train, Y_train)*100), decision_tree.score(valid_X_train, valid_Y_train)*100)

82.08955223880598
78.67564534231201 77.20706260032102


In [3377]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(valid_X_train, valid_Y_train)
Y_pred = random_forest.predict(valid_X_test)
random_forest.score(valid_X_train, valid_Y_train)
#acc_random_forest = round(random_forest.score(valid_X_train, valid_Y_train) * 100, 2)
#acc_random_forest

print(accuracy_score(valid_Y_test, Y_pred)* 100)

#a check to see if data overfitted- if these numbers are close together, then the data is not overfit 
print((random_forest.score(X_train, Y_train)*100), random_forest.score(valid_X_train, valid_Y_train)*100)

82.08955223880598
78.67564534231201 77.20706260032102


## Pick the Best Classifier and Create Submission 

Now that I have finished printing my accuracy, I have decided that I will use decision tree as my submission and winning classifier, as it consistently prints an accuracy score around 80. 

In [3378]:
final_pred = decision_tree
submission = pd.DataFrame({
    'Pclass': test_df['Pclass'],
    'Survived': final_pred
})
submission
submission.to_csv('submision.csv', index=False)

**Areas of Future Exploration:**

- I would have really loved to have achieved a higher accuracy score, and I think I could have done this by changing the weight for the various columns I ended up training my classifiers with
    - for example, because sex is the greatest determinate of survival, it would be great if I could find a way to have that column carry more weight overall. I think this could be done with neural networks, and I hope to explore this in the future! 
    

**Additional Notes:**

- I tried to combine different columns (other than just SibSp and Parch) into a combined data to maybe give more weight to one column over the other, as I thought that overlapping would boost my accuracy score
    - I tried a few different combinations, and ultimatley decided that it acctually greatly decreased my accuracy score
    - In the future, however, I hope to find out different ways to overlap data 

**Questions I still Have:**

- Why didn't combine work for my data set? (still a mystery)
- Julia ended up getting a higher accuracy score than mine: using sex, passenger Id, and fare
    - But when I tried just using the same features she used, my accuracy score greatly decreased
    - Why did that happen? 

**Acknowledgements:**

- Manav Sehgal greatly helped me with my code, and I used a lot of strategy and code from him. His notebook allowed me to analyze effectively and I learned many new classifiers from his notebook. 
- Ms. Sconyers greatly helped me navigate through errors in my code, and helped me strategize on which data I ended up using for training my classifiers. I couldn't have completed this lab without all of her help!

Links to Sources:

https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8 

https://www.kaggle.com/startupsci/titanic-data-science-solutions